In [3]:
import xarray as xr
import pickle 
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from eofs.xarray import Eof
from eofs.multivariate.iris import MultivariateEof
import iris
#import iris.plot as iplt
#import iris.quickplot as qplt
import datetime

anomalies_norm = xr.open_dataset("/home/srvx11/lehre/users/a1204101/AnalogMethod/data/ERA5/anom_all_1979-2018.nc",chunks={'time': 5000, 'lat':141, 'lon': 141})


In [ ]:
result = pd.DataFrame(columns=['search_day', 'found_day'])
N_list=np.array([])
var_f_list=np.array([])
for doy in range(1,367):   # Loop über alle 366 dayofyears wenn: range(1,367):
    print('berechne doy: ', doy) # = Target Day
    
    # Bestimmen der benötigten doy im Zeitfenster
    days = np.arange(doy-10,doy+11)
    days[days<=0] = days[days<=0]+366
    days[days>366] = days[days>366]-366
    
    # Ausschneiden der 21 Tage (in allen Jahren), Größe: 21 Tage x 40 Jahre x 141x141
    anomalies_i_q = anomalies_norm.q.where(anomalies_norm.q.dayofyear.isin(days),drop=True).to_iris()
    anomalies_i_r = anomalies_norm.r.where(anomalies_norm.r.dayofyear.isin(days),drop=True).to_iris()
    anomalies_i_msl = anomalies_norm.msl.where(anomalies_norm.msl.dayofyear.isin(days),drop=True).to_iris()
    
    # solver-Objekt erzeugen
    solver = MultivariateEof([anomalies_i_q,anomalies_i_r,anomalies_i_msl])
    
    # immer mehr PCs bis 90% erklärte Varianz:
    N = 1
    var_f = 0
    while var_f < .9:
        var_f = np.sum(solver.varianceFraction(neigs=N).data)
        N = N+1
    N_list = np.append(N_list,N)
    var_f_list = np.append(var_f_list,var_f)
    pcs = solver.pcs(npcs=N)
    pcs_x = xr.DataArray.from_iris(pcs) # Umwandeln der PCs in xarray
    del pcs    
        
    for year in np.arange(1979,2019): # Loop über die 40 Jahre
        
        # Ausschneiden der einen benötigten Analyse (= ein Tag), Größe: 141x141
        single_q = anomalies_norm.q.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).to_iris()
        single_r = anomalies_norm.r.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).to_iris()
        single_msl = anomalies_norm.msl.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).to_iris()
        
        if single_q.coord('time').points.size > 0:  # nur wenn der doy in dem Jahr existiert (doy 366 nur in Schaltjahren)
            
            # Berechnen der Pseudo-PCs für den einen Tag
            pseudo_pcs = solver.projectField([single_q, single_r, single_msl], neofs=N)
        
            # Minimieren der Norm (=Suchen des analogen Tags = 'found_day')
            found_idx=np.argmin(np.sum(np.sqrt((pcs_x.sel(time=~(pcs_x.time.dt.year == year)).data - pseudo_pcs.data)**2),axis=1)) # Index im pcs
        
            search_day = anomalies_norm.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).time[0].data # 'aktueller Tag'
            found_day = pcs_x.sel(time=~(pcs_x.time.dt.year == year)).isel(time=found_idx).time.data
            
            result = result.append(pd.DataFrame([[pd.to_datetime(search_day),pd.to_datetime(found_day)]],columns=['search_day', 'found_day']))

result = result.sort_values('search_day') # richtige Reihenfolge machen
#result.reset_index(drop=True) # nur formal

pickle.dump(result, open( "save_found_dates.p", "wb" ) )
# öffnen mit XXX = pickle.load( open( "save_found_dates.p", "rb" ) )
pickle.dump(N_list, open( "save_N_list.p", "wb" ) )
pickle.dump(var_f_list, open( "save_var_f_list.p", "wb" ) )
print(result)